<a href="https://colab.research.google.com/github/Satya-1729/Python_machine_learning/blob/main/evaluation_of_neural_networks_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


# reduce display decimal points to 2
# np.set_.printoptions(precision = 2)
np.set_printoptions(precision=2)

# supressing the annoying warning
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(0)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [7]:
def build_models():

  model_one = Sequential([
    Dense(units = 25, activation = 'relu'),
    Dense(units = 10, activation = 'relu'),
    Dense(units = 1, activation='linear')
  ])

  model_two = Sequential([
    Dense(units = 25, activation = 'relu'),
    Dense(units = 15, activation = 'relu'),
    Dense(units = 1, activation='linear')
  ])

  model_three = Sequential([
    Dense(units = 30, activation = 'relu'),
    Dense(units = 9, activation = 'relu'),
    Dense(units = 1, activation='linear')
  ])

  return [model_one, model_two, model_three]

x=build_models()
print(x.name())








[<Sequential name=sequential_3, built=False>, <Sequential name=sequential_4, built=False>, <Sequential name=sequential_5, built=False>]


In [8]:
# loading the binary dataset

df = pd.read_csv('/content/emails.csv')
df.head()




,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [9]:
# intializing the colums for target and train
x = df.iloc[:,0]
y = df.iloc[:,-1]

print(x[:5])
print(y[:5])


0    Subject: naturally irresistible your corporate...
1    Subject: the stock trading gunslinger  fanny i...
2    Subject: unbelievable new homes made easy  im ...
3    Subject: 4 color printing special  request add...
4    Subject: do not have money , get software cds ...
Name: text, dtype: object
0    1
1    1
2    1
3    1
4    1
Name: spam, dtype: int64


In [10]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer converting the words into its base form for e.g. running -> run
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
# Text preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase

    #sting punctuation has all the special characters in it !@#$%%^^.. all these
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation

    words = text.split() #it splits the that is came after translate

    # Remove stopwords(is,and,of etc) & lemmatize(baseform)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]

    return " ".join(words)


In [12]:
# Instead of range(len(x)), you can use enumerate(x), which gives both the index (i) and the element (val) directly:
x1 = x.copy()
for i, val in enumerate(x):
    x1[i] = preprocess_text(val)

In [13]:
print(x1[:5])

0    subject naturally irresistible corporate ident...
1    subject stock trading gunslinger fanny merrill...
2    subject unbelievable new home made easy im wan...
3    subject 4 color printing special request addit...
4    subject money get software cd software compati...
Name: text, dtype: object


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf_vectorizer = TfidfVectorizer()

x1_tfid = tfidf_vectorizer.fit_transform(x1)
x1_tfid = x1_tfid.toarray()


In [15]:
print(x1_tfid.shape)

(5728, 34605)


In [16]:
# now we will split the dataset into train test and cross-validation set
x_train, x_, y_train, y_ = train_test_split(x1_tfid, y, test_size = 0.4, random_state = 0)

# now splitting the remaining data into cross validation and test set
x_test, x_cv, y_test, y_cv = train_test_split(x_, y_, test_size = 0.5, random_state = 0)

# now deleting the variable
del x_,y_

print(x_train.shape,"\n")
print(y_train.shape,"\n")
print(x_test.shape,"\n")
print(y_test.shape,"\n")
print(x_cv.shape,"\n")
print(y_cv.shape,"\n")

(3436, 34605) 

(3436,) 

(1146, 34605) 

(1146,) 

(1146, 34605) 

(1146,) 



In [17]:
#now we will expand the dimensions of the dataset to 2d array cause all  the operation we do on 2d array


y_train = np.expand_dims(y_train, axis = 1)

y_test = np.expand_dims(y_test, axis = 1)

y_cv = np.expand_dims(y_cv, axis = 1)

In [18]:
#now we will check the dimensions
print(x_train.shape,"\n")
print(y_train.shape,"\n")
print(x_test.shape,"\n")
print(y_test.shape,"\n")
print(x_cv.shape,"\n")
print(y_cv.shape,"\n")

(3436, 34605) 

(3436, 1) 

(1146, 34605) 

(1146, 1) 

(1146, 34605) 

(1146, 1) 



In [19]:
scaler = StandardScaler(with_mean = False)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_cv = scaler.transform(x_cv)

In [20]:
print(x_test[0].shape)

(34605,)


In [21]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam


models = Sequential([
    Dense(units = 25, activation = 'relu'),
    Dense(units = 10, activation = 'relu'),
    Dense(units = 1, activation='linear')
])

models.compile(optimizer = 'adam', loss = 'BinaryCrossentropy')
models.fit(x_train, y_train,epochs = 10, batch_size = 32, validation_data = (x_cv, y_cv))

threshold =0.5
yhat = models.predict(x_train)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1,0)

train_error = np.mean(yhat!=y_train)
print(train_error)


Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 1.8610 - val_loss: 0.8105
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - loss: 0.2620 - val_loss: 0.7299
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.1327 - val_loss: 0.6903
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.1021 - val_loss: 0.7030
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.0884 - val_loss: 0.5848
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.1420 - val_loss: 0.6190
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.1041 - val_loss: 0.5571
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.0930 - val_loss: 0.5325
Epoch 9/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - loss: 0.0363 - val_loss: 0.5808
Epoch 10/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0792 - val_loss: 0.6182
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
0.004365541327124563


In [24]:
# evaluating the models

nn_train_error = []
nn_cv_error = []

# build the models

modelz = build_models()

for model in modelz:
  #setup the loss function and optimizer

  model.compile(
      loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
      optimizer = Adam(learning_rate = 0.01),
  )

  print(f'training{model.name}')

  #training the model
  model.fit(
      x_train,
      y_train,
      epochs = 10,
      verbose = 0
  )

  print("Done!")

  #setting the threshold for classification

  threshold = 0.5

  # record the fration of misclassified for training data
  yhat = model.predict(x_train)
  yhat = tf.math.sigmoid(yhat)
  yhat = np.where(yhat >= threshold, 1, 0)
  train_error = np.mean(yhat != y_train)
  nn_train_error.append(train_error)

  # record the fration of misclassified for validation data
  yhat = model.predict(x_cv)
  yhat = tf.math.sigmoid(yhat)
  yhat = np.where(yhat >= threshold, 1, 0)
  cv_error = np.mean(yhat != y_cv)
  nn_cv_error.append(cv_error)






trainingsequential_10
Done!
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
trainingsequential_11
Done!
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
trainingsequential_12
Done!
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [26]:
for model_num in range(len(nn_train_error)):
  print(f"Model{model_num + 1}")
  print(f"Train Error: {nn_train_error[model_num]}")
  print(f"CV Error: {nn_cv_error[model_num]}")

print(nn_train_error)
print(nn_cv_error)

Model1
Train Error: 0.0
CV Error: 0.030541012216404886
Model2
Train Error: 0.0
CV Error: 0.02530541012216405
Model3
Train Error: 0.0
CV Error: 0.02530541012216405
[0.0, 0.0, 0.0]
[0.030541012216404886, 0.02530541012216405, 0.02530541012216405]


In [27]:
#selecting the best model
model_num = 2

# computing the test error
yhat = modelz[model_num-1].predict(x_test)
yhat = tf.math.sigmoid(yhat)
yhat = np.where(yhat >= threshold, 1, 0)
nn_test_error = np.mean(yhat != y_test)
print(nn_test_error)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
0.030541012216404886
